# 가설과 추론
앞에서 배운 통계와 확률 이론으로 가설을 검정하고 통계적 추론을 해보자. 데이터와 데이터를 발생시킨 현상에 대한 가설을 세우고 검정을 한다.

## 7.1 통계적 가설검정
가설(hypothesis): "이 동전은 앞뒤가 나올 확률이 공평한 동전이다.", "데이터 과학자는 R보다 Python을 선호한다.", "닫기 버튼이 작아서 찾기 힘든 광고 창을 띄우면 사용자는 해당 사이트를 죽었다 깨어나도 들어가지 않을 것이다." 등과 같은 주장을 의미. \
데이터 통계치에 대한 얘기로 변환될 수 있다. 통계치들은 다양한 가정하에서 특정 분포에 대한 확룰변수의 관측치로 이해할 수 있고, 그런 가정들이 얼마나 타당한지 알 수 있게 해준다. \
고전적인 가설검정에서는 기본 입장을 나타내는 **귀무가설($H_0$, null hypothesis)**과 대비되는 입장을 나타내는 **대립가설($H_1$, alternative hypothesis)** 을 통계적ㅇ로 비교해서 귀무가설을 기각할지 말지 결정한다. 

## 7.2  예시: 동전 던지기
동전이 하나 있다. 이 동전이 공평한 동전인지 아닌지 검정하고 싶다. 이 동전에서 앞면이 나올 확률이 p라고 하면 동전이 공평하다는 의미의 'p = 0.5'이다는 귀무가설이되고 'p $\neq$ 0.5 이다'는 대립가설이 된다. \
동전을 n번 던져 앞면이 나온 횟수 X를 세는 것으로 검정을 진행, 동전 던지기는 각각 베르누이 분포를 다를 것이며, 이는 X가 이항분포를 따르는 확률변수라는 것을 의미한다. 이항분포는 정규 분포로 근사할 수 있다.

In [2]:
from typing import Tuple
import math

def normal_approximation_to_binomial(n: int, p: float) -> Tuple[float, float]:
    """Binomial(n,p)에 해당되는 mu(평균)와 sigma(표준편차) 계산"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

In [3]:
def normal_cdf(x: float, mu: float = 0, sigma: float = 1) -> float:
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

확률변수가 정규분포를 따른다는 가정하에 normal_cdf를 사용하면 실제 동전 던지기로부터 얻은 값이 구간 안(혹은 밖)에 존재할 확률을 계산할 수 있다.

In [17]:
# 누적 분포 함수는 확률변수가 특정 값보다 작을 확률을 나타낸다.
normal_probability_below = normal_cdf

# 만약 확률변수가 특정 값보다 작지 않다면, 특정 값보다 크다는 것을 의미한다.
def normal_probability_above(lo: float,
                            mu: float = 0,
                            sigma: float = 1) -> float:
    """mu(평균)와 sigma(표준편차)를 따르는 정규분포가 lo보다 클 확률"""
    return 1 - normal_cdf(lo, mu, sigma)

# 만약 확률변수가 hi보다 작고 lo보다 작지 않다면 확률변수는 hi와 lo 사이에 존재한다.
def normal_probability_between(lo: float,
                              hi: float,
                              mu: float = 0,
                              sigma: float = 1) -> float:
    """mu(평균)와 sigma(표준편차)를 따르는 정규분포가 lo와 hi 사이에 있을 확률"""
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# 만약 확률변수가 범위 밖에 존재한다면 범위 안에 존재하지 않다는 것을 의미한다.
def normal_probability_outside(lo: float,
                              hi: float,
                              mu: float = 0,
                              sigma: float = 1) -> float:
    """mu(평균)와 sigma(표준편차)를 따르는 정규분포가 lo와 hi 사이에 없을 확률"""
    return 1 - normal_probability_between(lo, hi, mu, sigma)

In [5]:
def inverse_normal_cdf(p: float,
                      mu: float = 0,
                      sigma: float = 1,
                      tolerance: float = 0.00001) -> float:
    """이진 검색을 사용해 역함수를 근사"""
    # 표준정규분포가 아니라면 표준정규분포로 변환
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)
    
    low_z = -10.0  # normal_cdf(-10)은 0에 근접
    hi_z = 10.0  # normal_cdf(10)은 1에 근접
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2  # 중간 값
        mid_p = normal_cdf(mid_z)  # 중간 값의 누적분포 값을 계산
        if mid_p < p:
            low_z = mid_z  # 중간 값이 너무 작다면 더 큰 값들을 검색
        else:
            hi_z = mid_z  # 중간 값이 너무 크다면 더 작은 값들을 검색
    
    return mid_z

반대로, 확률이 주어졌을 때 평균을 중심으로 하는 (대칭적인) 구간을 구할 수도 있다. 예를 들어 분포의 60%를 차지하는 평균 중심의 구간을 구하고 싶다면 양
쪽 꼬리 부분이 각각 분포의 20%를 차지하는 지점을 구하면 된다.

In [18]:
def normal_upper_bound(probability: float,
                      mu: float = 0,
                      sigma: float = 1) -> float:
    """P(Z <= z) = probability인 z 값을 반환"""
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability: float,
                           mu: float = 0,
                           sigma: float = 1) -> float:
    """P(Z >= z) = probability인 z 값을 반환"""
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability: float,
                           mu: float = 0,
                           sigma: float = 1) -> Tuple[float, float]:
    """
    입력한 probability 값을 포함하고,
    평균을 중심으로 대칭적인 구간을 반환
    """
    tail_probability = (1 - probability) / 2
    
    # 구간의 상한은 tail_probability 값 이상의 확률 값을 갖고 있다.
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    
    # 구간의 하한은 tail_probability 값 이하의 확률 값을 갖고 있다.
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    
    return lower_bound, upper_bound

In [19]:
# 동전을 1,000번 던져 보자(n = 1000). 가설이 맞다면 X는 대략 평균이 500 이고 표준편차가 15.8인 정규분포를 따를 것이다.
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)
print("mu_0:", mu_0, "sigma_0:", sigma_0)

mu_0: 500.0 sigma_0: 15.811388300841896


이제 제1종 오류를 얼마나 허용해 줄 것인지를 의미하는 유의수준(significance)을 결정해야 한다. **제 1종 오류**란 비록 $H_0$가 참이지만 $H_0$를 기각하는 'false positive(가양성)' 오류를 의미한다. 유의수준은 보통 5%나 1%로 설정하는 경우가 많은데, 여기서는 유의수준을 5%로 선택해 보자. \
다음의 코드에서 X가 주어진 범위를 벗어나면 귀무가설 $H_0$를 기각하는 가설검정을 고려해 보자.

In [20]:
# (469, 531)
lower_bound, upper_bound = normal_two_sided_bounds(0.95, mu_0, sigma_0)  # 유의수준을 5%로 선택함
print(lower_bound, upper_bound)

469.01026640487555 530.9897335951244


p가 정말로 0.5, 즉 $H_0$가 참이라면 X가 주어진 범위를 벗어날 확률은 우리가 원한 대로 5%밖에 되지 않을 것이다. 바꿔 말하면, 만약 $H_0$가 참이라면 이 가설검정은 20번 중 19번은 올바른 결과를 줄 것이다. \
한편 제 2종 오류를 범하지 않을 확률을 구하면 검정력(power)을 알 수 있다. **제 2종 오류**란 $H_0$가 거짓이지만 $H_0$를 기각하지 않는 오류를 의미.\
제 2종 오류를 측정하기 위해 먼저 $H_0$가 거짓이라는 것이 무엇을 의미하는지 알아볼 필요가 있다. (p가 0.5가 아니라는 말은 X의 분포에 관한 많은 것을 알려 주지는 않는다.) 예를 들어, p가 0.55, 즉 동전의 앞면이 나올 확률이 약간 편향되어 있다면 검정력은 다음과 같다.

In [22]:
# p가 0.5라고 가정할 때, 유의수준이 5%인 구간
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

# p = 0.55인 경우의 실제 평균과 표준편차
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

# 제 2종 오류란 귀무가설(H0)을 기각하지 못한다는 의미
# 즉, X가 주어진 구간 안에 존재할 경우를 의미
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability  # 0.887

print("mu_1:", mu_1, "sigma_1:", sigma_1)
print("type_2_probability:", type_2_probability)
print("power:", power)

mu_1: 550.0 sigma_1: 15.732132722552274
type_2_probability: 0.11345199870463285
power: 0.8865480012953671


In [ ]:
# p.100 한편 ...